In [1]:
import pandas as pd


import numpy as np
import pandas as pd

import seaborn as sns

import matplotlib.pyplot as plt
%matplotlib inline

from datetime import datetime

In [2]:
# path = '/home/hassan/lily/MLA/FDA/inpatient_full_simple.csv'
# inpatient = pd.read_csv(path).iloc[:,1:]
# inpatient.head()

In [7]:
path = '/data/public/MLA/VCHAMPS-Test/procedures_test.csv'
procedures = pd.read_csv(path,engine = 'python',on_bad_lines='warn')
procedures

Skipping line 18710187: unexpected end of data


,Unnamed: 0,Internalpatientid,Age at procedure,Procedure date,Procedure code,Procedure code description,State
0,112,100,52.559428,2014-04-19 11:18:05.0,99343,HOME VISIT FOR THE EVALUATION AND MANAGEMENT O...,New York
1,113,100,52.559428,2014-04-19 11:18:05.0,G0154,DIRECT SKILLED NURSING SERVICES OF A LICENSED ...,New York
2,114,100,52.753358,2014-06-29 08:27:24.0,97003,OCCUPATIONAL THERAPY EVALUATION,New York
3,115,100,52.753358,2014-06-29 08:27:24.0,97535,"SELF-CARE/HOME MANAGEMENT TRAINING (EG, ACTIVI...",New York
4,116,100,53.094739,2014-11-01 03:03:17.0,G0155,SERVICES OF CLINICAL SOCIAL WORKER IN HOME HEA...,New York
...,...,...,...,...,...,...,...
18710180,89979360,64640,77.455704,2008-04-01 01:25:53.0,99205,OFFICE OR OTHER OUTPATIENT VISIT FOR THE EVALU...,Florida
18710181,89979361,64640,78.967937,2009-10-05 18:44:49.0,99371,TELEPHONE CALL BY A PHYSICIAN TO PATIENT OR FO...,New Jersey
18710182,89979362,64640,79.269506,2010-01-24 00:06:20.0,90658,"INFLUENZA VIRUS VACCINE, TRIVALENT (IIV3), SPL...",New Jersey
18710183,89979363,64640,79.547951,2010-05-05 18:37:44.0,85610,PROTHROMBIN TIME;,New Jersey


In [8]:
procedures = procedures.drop_duplicates(subset = ['Internalpatientid', 'Procedure date','Procedure code description'],keep = 'first')

In [9]:
# inpatient_date = inpatient[['Internalpatientid','Discharge date']]
# inpatient_date = inpatient_date.drop_duplicates(keep = 'first')
# inpatient_date

In [10]:
# # Filter procedure date that's before discharge date
# inpatient_pro = procedures.merge(inpatient_date, how = 'left', on = 'Internalpatientid')
# inpatient_pro2 = inpatient_pro.loc[inpatient_pro['Procedure date'] <= inpatient_pro['Discharge date']]
# inpatient_pro2 

In [11]:
# Calculate total procedures before last admission for each patient
procedures['total_procedure'] = procedures['Procedure code description'].groupby(procedures['Internalpatientid']).transform('count')

/tmp/ipykernel_262961/750668648.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  procedures['total_procedure'] = procedures['Procedure code description'].groupby(procedures['Internalpatientid']).transform('count')


In [12]:
# Calculate total procedures that's surgery before last admission for each patient
surgery_pro = procedures.loc[procedures['Procedure code description'].str.contains('SURG')]
surgery_pro['num_surgery_pro'] = surgery_pro['Procedure code'].groupby(surgery_pro['Internalpatientid']).transform('count')

/tmp/ipykernel_262961/3501479404.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  surgery_pro['num_surgery_pro'] = surgery_pro['Procedure code'].groupby(surgery_pro['Internalpatientid']).transform('count')


In [13]:
procedures['Internalpatientid'].nunique()

26894

In [14]:
columns = ['Internalpatientid', 'total_procedure']
df = procedures[columns]
df = df.groupby('Internalpatientid').mean().reset_index(drop = False)
df = df.merge(surgery_pro[['Internalpatientid','num_surgery_pro']], how = 'left',on = 'Internalpatientid')
df['num_surgery_pro'] = df['num_surgery_pro'].fillna(0)
df = df.groupby('Internalpatientid').mean().reset_index(drop = False)
df

,Internalpatientid,total_procedure,num_surgery_pro
0,6,767.0,3.0
1,12,247.0,0.0
2,17,338.0,1.0
3,22,259.0,0.0
4,25,881.0,9.0
...,...,...,...
26889,169037,783.0,0.0
26890,169045,40.0,0.0
26891,169058,86.0,1.0
26892,169059,520.0,0.0


In [15]:
df.to_csv('/home/daisy/FDA_Dataset/inpatient_procedures_clean_cvd_mortality_test.csv')